In [1]:
import gym 
import torch
from collections import deque
import random 
import copy
from torch.autograd import Variable
env = gym.envs.make("CartPole-v0")

/home/prayash/.local/lib/python3.10/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [2]:
class DQN():
    def __init__(self, n_state,n_action,  n_hidden=50, lr=0.05):
        self.criterion = torch.nn.MSELoss()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(n_state, n_hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(n_hidden, n_action)
        )
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr)

        self.model_target = copy.deepcopy(self.model)

    def target_predict(self, s):
        with torch.no_grad():
            return self.model_target(torch.Tensor(s))

    def copy_target(self):
        self.model_target.load_state_dict(self.model.state_dict())

    def replay(self, memory, replay_size, gamma ):
        if len(memory) >= replay_size:
            replay_data = random.sample(memory, replay_size)
            states= []
            td_targets =[]
            for state, action, next_state, reward, is_done in replay_data:
                states.append(state)
                q_values = self.predict(state).tolist()
                if is_done:
                    q_values[action]= reward
                else:
                    q_values_next = self.target_predict(next_state).detach()
                    q_values[action] = reward + gamma*torch.max(q_values_next).item()

                td_targets.append(q_values)

            self.update(states, td_targets)
    def update(self, s,y):
        """
        Update the weights of the DQN given a training sample
        @param s : state
        @param y: target value
        """

        y_pred = self.model(torch.Tensor(s))
        loss = self.criterion(y_pred, Variable(torch.Tensor(y)))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def predict(self, s ):
        with torch.no_grad():
            return self.model(torch.Tensor(s))

In [3]:
def gen_epsilon_greedy_policy(estimator, epsilon , n_action):
    def policy_function(state):
        if random.random() < epsilon:
            return random.randint(0, n_action -1 )

        else :
            q_values = estimator.predict(state)
            return torch.argmax(q_values).item()

    return policy_function


In [4]:
def q_learning(env, estimator, n_episode, replay_size, target_update=10, gamma = 1.0, epsilon =0.1, epsilon_decay=0.99):
    for episode in range(n_episode):
        if episode % target_update ==0:
            estimator.copy_target()
        
        policy = gen_epsilon_greedy_policy(
            estimator, epsilon, n_action
        )
        state, _ = env.reset()
        is_done = False
        while not is_done:
            action = policy(state)
            next_state, reward, is_done,_,_ = env.step(action)
            total_reward_episode[episode] +=reward
            memory.append((state, action,next_state, reward,is_done))

            if is_done:
                break
            
            estimator.replay(memory, replay_size, gamma)
            state = next_state

        epsilon = max(epsilon* epsilon_decay, 0.01)

In [5]:

n_state = env.observation_space.shape[0]
n_action = env.action_space.n
n_episode = 600
last_episode = 200



In [6]:
n_hidden_options = [30,40]
lr_options = [0.001, 0.003]
replay_size_options = [20,25]
target_update_options = [30, 35]


In [7]:
for n_hidden in n_hidden_options:
    for lr in lr_options:
        for replay_size in replay_size_options:
            for target_update in target_update_options:
                env.action_space.seed(1)
                random.seed(1)
                torch.manual_seed(1)
                dqn = DQN(n_state, n_action, n_hidden, lr)
                memory = deque(maxlen=10000)
                total_reward_episode =[0] * n_episode
                q_learning(env, dqn, n_episode, replay_size, target_update, gamma=.9, epsilon=1)
                print(n_hidden, lr, replay_size, target_update, sum(total_reward_episode[-last_episode:])/last_episode)

/tmp/ipykernel_2079/3373090565.py:44: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  y_pred = self.model(torch.Tensor(s))


30 0.001 20 30 219.085


KeyboardInterrupt: 